# Tutorial Fase 2: Infraestructura NLP para Análisis Sociológico

**Proyecto**: *De la billetera móvil al Actor Social (Lisbeth)*
**Objetivo**: Establecer los cimientos computacionales para analizar cómo evoluciona el significado de "Yape" en la prensa peruana (2016-2023).

---

Este notebook explica paso a paso la **metodología técnica** que estamos implementando. Para esta demostración, **crearemos un entorno aislado (sandbox)** con datos de muestra, asegurando que la ejecución sea rápida y no afecte a los modelos o datos reales del proyecto.

### Conceptos Clave
1.  **DAPT (Domain-Adaptive Pretraining)**: Enseñamos al modelo el "habla peruana".
2.  **Subword Pooling**: Reconstruimos matemáticamente palabras compuestas ("Yapear").
3.  **Embeddings Contextuales**: Capturamos el significado en uso.

## 1. Configuración del Entorno y Sandbox
Configuramos las rutas y preparamos una carpeta temporal `demo_sandbox` para nuestros experimentos.

In [ ]:
import sys
import os
import shutil
import pandas as pd
import torch
import glob

# 1. Ajuste robusto del PATH para importar src
# Si estamos en 'notebooks/', subimos un nivel. Si estamos en root, nos quedamos.
if os.path.basename(os.getcwd()) == "notebooks":
    PROJECT_ROOT = os.path.abspath("..")
else:
    PROJECT_ROOT = os.getcwd()

if PROJECT_ROOT not in sys.path:
    sys.path.append(PROJECT_ROOT)

# Importamos módulos del proyecto
from src.nlp.model import LisbethModel
from src.nlp.dapt import dapt
from src.nlp.extract import extract_embeddings

# 2. Crear entorno Sandbox (Carpetas temporales)
SANDBOX_DIR = os.path.join(PROJECT_ROOT, "demo_sandbox")
DATA_DEMO = os.path.join(SANDBOX_DIR, "data")
MODELS_DEMO = os.path.join(SANDBOX_DIR, "models")

# Limpiamos y recreamos el sandbox
if os.path.exists(SANDBOX_DIR):
    shutil.rmtree(SANDBOX_DIR)
os.makedirs(DATA_DEMO, exist_ok=True)
os.makedirs(MODELS_DEMO, exist_ok=True)

print(f"Entorno configurado en: {SANDBOX_DIR}")
print(f"Torch disponible: {torch.cuda.is_available()}")

## 2. Preparación de Datos de Muestra
En lugar de procesar los gigabytes de datos reales, tomaremos una pequeña muestra de los CSV originales para demostrar el funcionamiento.

In [ ]:
# Localizar datos reales
REAL_DATA_DIR = os.path.join(PROJECT_ROOT, "data")
csv_files = glob.glob(os.path.join(REAL_DATA_DIR, "yape_*.csv"))

if csv_files:
    # Tomamos el primer archivo y leemos solo 20 filas
    source_csv = csv_files[0]
    print(f"Creando muestra desde: {os.path.basename(source_csv)}")
    
    df_sample = pd.read_csv(source_csv, nrows=20)
    
    # Guardamos como 'yape_demo.csv' en la carpeta demo
    sample_csv_path = os.path.join(DATA_DEMO, "yape_demo.csv")
    df_sample.to_csv(sample_csv_path, index=False)
    
    # Creamos también un mini corpus de texto para DAPT
    corpus_sample_path = os.path.join(DATA_DEMO, "corpus_sample.txt")
    with open(corpus_sample_path, "w") as f:
        if "plain_text" in df_sample.columns:
            for text in df_sample["plain_text"].dropna():
                f.write(str(text).replace("\n", " ") + "\n")
    print("Datos de muestra generados exitosamente.")
else:
    print("ADVERTENCIA: No se encontraron datos reales en 'data/'. Se usarán datos sintéticos.")
    # Crear dato sintético si no hay CSVs
    corpus_sample_path = os.path.join(DATA_DEMO, "corpus_sample.txt")
    with open(corpus_sample_path, "w") as f:
        f.write("El aplicativo Yape permite yapear dinero fasilmente.\n" * 10)
    sample_csv_path = os.path.join(DATA_DEMO, "yape_demo.csv")
    pd.DataFrame({"plain_text": ["Yape es genial."]*10, "published_at": ["2023-01-01"]*10, "newspaper": ["Demo"]*10}).to_csv(sample_csv_path, index=False)

## 3. Tokenización y Subword Pooling
Probamos cómo el modelo maneja palabras clave como "Yapear". Usamos `xlm-roberta-base` para esta demo por su ligereza y compatibilidad.

In [ ]:
MODEL_NAME = "xlm-roberta-base"
model = LisbethModel(model_name=MODEL_NAME)

frase = "Voy a yapear el pago."
tokens = model.tokenizer.tokenize(frase)

print(f"Frase: '{frase}'")
print(f"Tokens: {tokens}")

# Extracción con pooling automático
try:
    embeddings = model.extract_embedding(frase, "yapear")
    if embeddings:
        print(f"Vector generado correctamente. Dimensión: {embeddings[0].shape}")
    else:
        print("No se pudo extraer el vector.")
except Exception as e:
    print(f"Error en extracción: {e}")

## 4. Ejecución del Pipeline Reducido

### A. Domain Adaptive Pretraining (Simulado)
Entrenamos el modelo por 1 sola época usando nuestro mini-corpus. El resultado se guarda en `demo_sandbox/models`.

In [ ]:
try:
    print("Entrenando modelo adaptado (Demo)...")
    dapt(MODEL_NAME, corpus_sample_path, MODELS_DEMO, epochs=1)
    print(f"✅ Modelo guardado en: {MODELS_DEMO}")
except Exception as e:
    print(f"❌ DAPT falló (esperable si falta memoria/gpu en entorno limitado): {e}")

### B. Extracción de Embeddings (Multi-Keyword)
Usamos el script `extract.py` para procesar nuestro CSV de muestra y buscar "Yape", "Yapear", etc.

In [ ]:
OUTPUT_PARQUET = os.path.join(DATA_DEMO, "embeddings_output.parquet")
KEYWORDS = ["Yape", "Yapear", "Plin"]

print(f"Extrayendo embeddings para {KEYWORDS}...")

try:
    extract_embeddings(DATA_DEMO, OUTPUT_PARQUET, keywords=KEYWORDS, model_name=MODEL_NAME)
    
    if os.path.exists(OUTPUT_PARQUET):
        df_results = pd.read_parquet(OUTPUT_PARQUET)
        print(f"\n✅ Extracción completada. Vectores obtenidos: {len(df_results)}")
        print(df_results.head(3))
    else:
        print("⚠️ No se encontraron las palabras clave en la muestra de datos.")
        
except Exception as e:
    print(f"Error en extracción masiva: {e}")

## 5. Limpieza (Opcional)
Podemos eliminar los datos temporales.

In [ ]:
# shutil.rmtree(SANDBOX_DIR)
print("Demo finalizada. Los archivos temporales están en 'demo_sandbox'.")